In [8]:
import torch 
import cv2 
import numpy as np 
from models import DeepPixBis
import os 
import time
from preprocessing import preprocess_video
import torch.nn.functional as F 
import csv

In [9]:
def write_predict_file(file_name='./result/jupyter_submission.csv', result=None, mode = 'time'):
    with open(file_name, 'w') as csv_file:
        writer = csv.writer(csv_file)
        if mode == 'time':
            writer.writerow(('fname', 'time'))
        else:
            writer.writerow(('fname','liveness_score'))
        for key, value in result.items():
            writer.writerow([key, value])

In [2]:
def load_model(net, path):
    if path is not None and path.endswith(".ckpt"):
        print(path)
        state_dict = torch.load(path, map_location='cpu')

        if "state_dict" in state_dict:
            state_dict = state_dict["state_dict"]
        compatible_state_dict = {}
        for k, v in state_dict.items():
            k = k[4:]
            compatible_state_dict[k] = v

        net.load_state_dict(compatible_state_dict)

    return net

In [3]:
model =  DeepPixBis(encoder_name='resnet18', num_classes=2, phase='test')
model = load_model(model, './weights/best-epoch=109-val_acc=0.99.ckpt')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

/home/vietlq4/miniconda3/envs/py37/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/vietlq4/miniconda3/envs/py37/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


./weights/best-epoch=109-val_acc=0.99.ckpt


DeepPixBis(
  (encoders): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1

In [28]:
data_path = '/home/vietlq4/Downloads/public_test_2/videos'
test_cases = sorted(os.listdir(data_path), key= lambda x: int(x.replace('.mp4','')))

In [29]:
all_predicted_time = {}
all_result = {}
for file_name in test_cases:
    t1 = time.time()
    input_ = preprocess_video(data_path,file_name, device)
    input_ = input_.unsqueeze(0)
    out_map, out = model(input_)
    out , out_map = out.detach(), out_map.detach()
    out = F.softmax(out, dim=1).detach().cpu().numpy()
    out = float(out[:,1])
    out_map = np.mean(out_map.numpy())
    final_out = (out + out_map) / 2
    t2 = time.time()
    predicted_time = int(t2*1000 - t1*1000)
    all_result[file_name] = final_out
    all_predicted_time[file_name] = predicted_time

try:
   if not os.path.exists(os.path.dirname('./result/jupyter_submission.csv')):
       os.makedirs(os.path.dirname('./result/jupyter_submission.csv'))
except OSError as err:
   print(err)
write_predict_file(file_name='./result/jupyter_submission.csv', result = all_result, mode='predict')
write_predict_file(file_name='./result/time_submission.csv', result=all_predicted_time, mode='time')

        

/home/vietlq4/miniconda3/envs/py37/lib/python3.7/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
